In [1]:
'''
tcloud中用销售订单生成税局发票模板
1. 分别按每个销售订单汇总，再按存货编码汇总，批量生成发票模板，存于同一文件夹下
2. 分别按每个客户汇总，再按存货编码汇总，批量生成发票模板，存于同一文件夹下
3. 选择税率
全部自动生成


'''
import os
import re
import easygui
import openpyxl
import numpy as np
import pandas as pd
import shutil

In [2]:
def getPivot(df):
    bianba_fenlei = dict(zip(df['存货编码'],df['存货分类']))
    bianba_mingchen = dict(zip(df['存货编码'],df['存货名称']))
    bianba_daima = dict(zip(df['存货编码'],df['存货代码']))
    bianba_hanlian = dict(zip(df['存货编码'],df['件含量']))
    pivot = df.pivot_table(index = '存货编码',values = ['数量','数量（件）','含税金额'] ,aggfunc = 'sum')
    pivot = pivot.reset_index()
    pivot = pivot.assign(fenlei = pivot['存货编码'].map(bianba_fenlei))
    pivot = pivot.assign(mingchen = pivot['存货编码'].map(bianba_mingchen))
    pivot = pivot.assign(daima = pivot['存货编码'].map(bianba_daima))
    pivot = pivot.assign(hanliang = pivot['存货编码'].map(bianba_hanlian))
    dic = dict(zip(['fenlei','mingchen','daima','hanliang'],['存货分类','存货名称','存货代码','件含量']))
    pivot = pivot.rename(columns =  dic)
    guige_qian = []
    guige_hou = []
    for i in pivot['存货名称'].to_list():
        qian0,hou0 = guige(i)
        guige_qian.append(qian0)
        guige_hou.append(hou0)
    pivot = pivot.assign(qian = guige_qian)
    pivot = pivot.assign(hou = guige_hou)
    return pivot

def getFapiaoBen(d,shuilu,shuliang_fangsi):
    
    d['项目名称'] = d['存货编码'] +  d['hou']
    d['项目名称'] = d['项目名称'].str.replace('运费-','运费')
    # d['项目名称'] = d['项目名称'].str.split('-').str[0]
    d['商品和服务税收分类编码'] = '1060202010000000000'
    d['规格型号'] = d['qian']
    if shuliang_fangsi == '本数':
        d['单位'] = '本'
        d['商品数量'] = d['数量']
    else :
        d['单位'] = '件'
        d['商品数量'] = d['数量（件）']
        
    d['商品单价'] = ''
    d['金额'] = d['含税金额']
    d['税率'] = shuilu
    d['折扣金额'] = ''
    d['优惠政策类型'] = ''
    d = d[['项目名称',
     '商品和服务税收分类编码',
     '规格型号',
     '单位',
     '商品数量',
     '商品单价',
     '金额',
     '税率',
     '折扣金额',
     '优惠政策类型',]]
    return d

def getFapiaoMoban(gongsi,shuliang_fangsi):
    filename=''.join(['发票模板-',shuliang_fangsi,'-',gongsi,'.xlsx'])
    newname = os.path.join(path1,filename)
    shutil.copyfile(fname_fapiao, newname)
    return newname

def getFapiaoMobanDingdanhao(gongsi,shuliang_fangsi,dingdanhao):
    filename=''.join(['发票模板-',gongsi,'-',shuliang_fangsi,'-',dingdanhao,'.xlsx'])
    newname = os.path.join(path1,filename)
    shutil.copyfile(fname_fapiao, newname)
    return newname

def fengefu(string):
    num = len(string.split('-'))
    if num  == 3 :
        string = '-'.join([string.split('-')[0],string.split('-')[1]])
    elif num == 2 :
        if len(string.split('-')[0]) <= 4:
            string = string
        else :
            string = string.split('-')[0]
    else :
        string = string
    return string

def guige(string):
    if ('型' in string) and ('页' in string):
        qian0 = string.split('型')[0] + '型'
        hou0 =  string.split('型')[1] 
    elif ('型' in string) or ('页' in string):
        if '型' in string :
            qian0 = string.split('型')[0] + '型'
            hou0 =  string.split('型')[1] 
        else :
            qian0 = string.split('页')[0] + '页'
            hou0 =  string.split('页')[1] 
    else :
        qian0 = ''
        hou0 = ''
    return qian0,hou0    
            
    
    

In [3]:
path = r'F:\a00nutstore\008\zw08\莱新订单20241029'
os.chdir(path)
#莱新销售订单超5000条，不能一次导出，分三次导出，并分别存于同一文件下，先将它们合并
data = []
for i in os.listdir(path):
    if  os.path.isfile(i):
        i = os.path.join(path,i)
        df = pd.read_excel(i)
        data.append(df)
    else :
        continue
   

df_xiaoshou0 = pd.concat(data)
df_xiaoshou1 = df_xiaoshou0.loc[df_xiaoshou0['单据执行状态'] != '合计']
lst1 = ['单据编号',
  '单据日期',
 '含税总金额',
 '存货名称',
 '存货分类',
 '存货编码',
  '存货代码',
  '数量',
  '件含量',
 '数量（件）',
  '含税单价',
 '含税金额',
        '客户']
df_xiaoshou2 = df_xiaoshou1[lst1]
df_xiaoshou3 = df_xiaoshou2.copy()
s = []
for i in df_xiaoshou2['存货编码'].to_list():
    j = fengefu(i)
    s.append(j)
df_xiaoshou3['存货编码'] = s

In [29]:
#对存货编码进一步处理，账本类变为账本+存货名称按'k'分列后的后面部分
def chuliZhangben(str1,str2):
    try:
        str1 = int(str1)
    except:
        str1 = str1
    if isinstance(str1,int):
        if 'K' in str2:
            qian,hou = str2.split('K')
        elif 'k' in str2:
            qian,hou = str2.split('k')
        else :
            qian = ''
            hou = str2
            
        str1 = str(str1) + hou
        str2 = str2
    else:
        str1 = str1
        str2 = str2
    newbianma = str1
    return newbianma
newbianmas = []
for index,row in dff.iterrows():
    bianma = row['存货编码']
    name = row['存货名称']
    newbianma = chuliZhangben(bianma,name)
    newbianmas.append(newbianma)
df_xiaoshou3['存货编码'] = newbianmas

In [30]:
#选择税率
shuilu = easygui.choicebox(msg = '请选择税率',choices = [0.13,0.01,0.02])
shuilu  = float(shuilu)
#选择数量的开具方式，件数or本数
shuliang_fangsi = easygui.choicebox(msg = '请选择数量开具方式',choices = ['件数','本数'])
# #选择发票开具方式
# fapiao_fangsi = easygui.choicebox(msg = '请选择发票开具方式',choices = ['按客户和存货编码','按销售订单和存货编码','按客户汇总'])

In [31]:
#读取发票模板
fname_fapiao = r"F:\repos\fish\发票模板.xlsx"
df_fapiao = pd.read_excel(fname_fapiao,header = 2,dtype = {'商品和服务税收分类编码':'str'})
df_fapiao

,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型


In [32]:
#按照发票开具方式，及数量选择方式，生成对应的文件夹
path1 = f'发票模板-按客户和存货编码-{shuliang_fangsi}'
path1 = os.path.join(path,path1)
if not os.path.exists(path1):
    try:
        os.mkdir(path1)
    except:
        pass

gp = df_xiaoshou3.groupby('客户')
for k,v  in gp:
    gongsi = k
    newname = getFapiaoMoban(gongsi,shuliang_fangsi)
    pivot = getPivot(v)
    # pivot1 = chuliMingchen(pivot)
    fapiao = getFapiaoBen(pivot,shuilu,shuliang_fangsi)
    with pd.ExcelWriter(newname, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
        fapiao.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)

#按照发票开具方式，及数量选择方式，生成对应的文件夹
path1 = f'发票模板-按销售订单和存货编码-{shuliang_fangsi}'
path1 = os.path.join(path,path1)
if not os.path.exists(path1):
    try:
        os.mkdir(path1)
    except:
        pass

gp = df_xiaoshou3.groupby('单据编号')
for k,v  in gp:
    dingdanhao = k
    gongsi = v['客户'].to_list()[0]
    newname = getFapiaoMobanDingdanhao(gongsi,dingdanhao,shuliang_fangsi)
    print(newname)
    pivot = getPivot(v)
    # pivot1 = chuliMingchen(pivot)
    fapiao = getFapiaoBen(pivot,shuilu,shuliang_fangsi)
    with pd.ExcelWriter(newname, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
        fapiao.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)

#按照发票开具方式，及数量选择方式，生成对应的文件夹
path1 = f'发票模板-按客户汇总-{shuliang_fangsi}'
path1 = os.path.join(path,path1)
if not os.path.exists(path1):
    try:
        os.mkdir(path1)
    except:
        pass
     
gp = df_xiaoshou3.groupby('客户')
for k,v  in gp:
    gongsi = k
    newname = getFapiaoMoban(gongsi,shuliang_fangsi)
    v1 = v.sum().T.to_frame().T
    v1.loc[0,'存货名称'] = '*印刷品*本册'
    v1.loc[0,'存货分类'] = '*印刷品*本册'
    v1.loc[0,'存货编码'] = '*印刷品*本册'
    v1.loc[0,'存货代码'] = '*印刷品*本册'
    pivot = getPivot(v1)
    # pivot1 = chuliMingchen(pivot)
    fapiao = getFapiaoBen(pivot,shuilu,shuliang_fangsi)
    with pd.ExcelWriter(newname, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
        fapiao.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)


    


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\work

F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-河南抬头喜办公用品有限公司S-SO-2024-02-1182-件数.xlsx


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-武汉开先文化用品有限责任公司-SO-2024-02-1184-件数.xlsx


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


...

C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-金牛区昌宏优品商贸部-SO-2024-02-1571-件数.xlsx
F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-成都杨玲-SO-2024-02-1572-件数.xlsx
F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-武汉市热脉文体用品有限公司-SO-2024-02-1573-件数.xlsx


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-武汉米圣贸易有限公司-SO-2024-02-1574-件数.xlsx
F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-河南抬头喜办公用品有限公司S-SO-2024-02-1575-件数.xlsx


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\work

In [7]:

    
    
    
    
    
    pivot
    
    
    
    
    


,存货编码,含税金额,数量,数量（件）,存货分类,存货名称,存货代码,件含量,qian,hou,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型
0,*印刷品*本册,6791.45,4892,23.0,*印刷品*本册,*印刷品*本册,*印刷品*本册,4292.0,,,*印刷品*本册,1060202010000000000,,件,23.0,,6791.45,0.13,,


In [8]:
gp

In [9]:
df00 = pd.DataFrame(gp)
df00

,0,1
0,上海流金岁月商贸有限公司,单据编号 单据日期 含税总金额 ...
1,临沂博涵文化用品有限公司,单据编号 单据日期 含税总金额 ...
2,临沂巧纳金纸品有限公司,单据编号 单据日期 含税总金额 ...
3,临沂李加朋,单据编号 单据日期 含税总金额 ...
4,义乌市博迅文体用品有限公司,单据编号 单据日期 含税总金额 ...
...,...,...
99,阜阳俊武文体商贸有限公司,单据编号 单据日期 含税总金额 ...
100,陕西富明文体用品有限公司,单据编号 单据日期 含税总金额 ...
101,青海一诺商贸有限公司,单据编号 单据日期 含税总金额 ...
102,鹤岗市超东选商贸有限公司,单据编号 单据日期 含税总金额 ...


In [10]:
df_xiaoshou3

,单据编号,单据日期,含税总金额,存货名称,存货分类,存货编码,存货代码,数量,件含量,数量（件）,含税单价,含税金额,客户
0,SO-2024-02-1418,2024-10-16,14064.05,A540型卡面无线胶订本,无线胶装软抄,N0671,抄本,3200,320.0,10.0,0.8352,2672.64,海南新宇晨办公用品有限公司
1,SO-2024-02-1418,2024-10-16,14064.05,A560型卡面无线胶订本,无线胶装软抄,N0672,抄本,1440,240.0,6.0,1.1712,1686.53,海南新宇晨办公用品有限公司
2,SO-2024-02-1418,2024-10-16,14064.05,A580型卡面无线胶订本,无线胶装软抄,N0673,抄本,540,180.0,3.0,1.5456,834.62,海南新宇晨办公用品有限公司
3,SO-2024-02-1418,2024-10-16,14064.05,A5100型卡面无线胶订本,无线胶装软抄,N0674,抄本,560,140.0,4.0,1.8912,1059.07,海南新宇晨办公用品有限公司
4,SO-2024-02-1418,2024-10-16,14064.05,A540型牛卡无线胶订本,无线胶装软抄,N0681,抄本,1600,320.0,5.0,0.8352,1336.32,海南新宇晨办公用品有限公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,SO-2024-02-1419,2024-10-15,47469.60,16K银行存款日记账,纸面账簿,6003,账本,300,60.0,5.0,5.5440,1663.20,品壹佰河北商贸有限公司
2160,SO-2024-02-1419,2024-10-15,47469.60,A480型牛皮无线胶订本,无线胶装软抄,N0697,抄本,100,100.0,1.0,3.3840,338.40,品壹佰河北商贸有限公司
2161,SO-2024-02-1419,2024-10-15,47469.60,A4100型牛皮无线胶订本,无线胶装软抄,N0698,抄本,80,80.0,1.0,4.3710,349.68,品壹佰河北商贸有限公司
2162,SO-2024-02-1419,2024-10-15,47469.60,16K现金日记账,纸面账簿,6105,账本,200,40.0,5.0,7.8720,1574.40,品壹佰河北商贸有限公司


In [12]:
df_xiaoshou3.to_excel('beforr_gp.xlsx')

In [8]:
dff = df_xiaoshou3.copy()

In [9]:
dff

,单据编号,单据日期,含税总金额,存货名称,存货分类,存货编码,存货代码,数量,件含量,数量（件）,含税单价,含税金额,客户
0,SO-2024-02-1418,2024-10-16,14064.05,A540型卡面无线胶订本,无线胶装软抄,N0671,抄本,3200,320.0,10.0,0.8352,2672.64,海南新宇晨办公用品有限公司
1,SO-2024-02-1418,2024-10-16,14064.05,A560型卡面无线胶订本,无线胶装软抄,N0672,抄本,1440,240.0,6.0,1.1712,1686.53,海南新宇晨办公用品有限公司
2,SO-2024-02-1418,2024-10-16,14064.05,A580型卡面无线胶订本,无线胶装软抄,N0673,抄本,540,180.0,3.0,1.5456,834.62,海南新宇晨办公用品有限公司
3,SO-2024-02-1418,2024-10-16,14064.05,A5100型卡面无线胶订本,无线胶装软抄,N0674,抄本,560,140.0,4.0,1.8912,1059.07,海南新宇晨办公用品有限公司
4,SO-2024-02-1418,2024-10-16,14064.05,A540型牛卡无线胶订本,无线胶装软抄,N0681,抄本,1600,320.0,5.0,0.8352,1336.32,海南新宇晨办公用品有限公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3321,SO-2024-02-1419,2024-10-15,47469.60,16K银行存款日记账,纸面账簿,6003,账本,300,60.0,5.0,5.5440,1663.20,品壹佰河北商贸有限公司
3322,SO-2024-02-1419,2024-10-15,47469.60,A480型牛皮无线胶订本,无线胶装软抄,N0697,抄本,100,100.0,1.0,3.3840,338.40,品壹佰河北商贸有限公司
3323,SO-2024-02-1419,2024-10-15,47469.60,A4100型牛皮无线胶订本,无线胶装软抄,N0698,抄本,80,80.0,1.0,4.3710,349.68,品壹佰河北商贸有限公司
3324,SO-2024-02-1419,2024-10-15,47469.60,16K现金日记账,纸面账簿,6105,账本,200,40.0,5.0,7.8720,1574.40,品壹佰河北商贸有限公司


['N0671',
 'N0672',
 'N0673',
 'N0674',
 'N0681',
 'N0682',
 'N0684',
 'FYP1660',
 'N0578',
 'N0568',
 'N0569',
 'N0577',
 'N0007',
 'RFZY-3260N',
 'N0566',
 'N0576',
 'N0670',
 'RJL-32150',
 'RJL-32150',
 'RJL-1680',
 'RJL-1680',
 'N0785',
 'N0783',
 'N0784（300格）',
 'RL-32150',
 'RL-32150',
 'EXLA5100Pa',
 'EXWGA5100Pa',
 'N0651',
 'DJL16150B',
 'ETTHA420N',
 'N0527',
 'N0528',
 'N0565',
 'N0821',
 'N0422',
 'N0423',
 'N0683',
 'N0430',
 'DL-3280',
 'N0697',
 'N0698',
 'N0826',
 'DB101',
 'DB102',
 'DB103',
 'DB104',
 'DB105',
 'DB302',
 'DB303',
 'DB304',
 'DB306',
 'DB307',
 'DB308',
 'DB309',
 'DJL16200B',
 'DJL16200B',
 'J1602',
 'N0822',
 'N0517',
 'N0516办',
 'XJ402S',
 'N0682',
 'N0683',
 'N0697',
 'N0698',
 'RJL-32150',
 'DJL80混装',
 'DJL64混装',
 'N0821',
 'N0422',
 'N0423',
 'N0430',
 'N0526',
 'N0527',
 'N0528',
 'N0529',
 'RFTZ-3260N',
 'RFHP-3260N',
 'EJY32150',
 'EJL32150',
 'N0697',
 'N0698',
 'N0002',
 '3013报销凭证粘贴单',
 'N0422',
 'N0423',
 'N0424',
 'N0425',
 'N0517',
 'N057

In [1]:
import pandas as pd

In [20]:
df0 = pd.read_excel(r"F:\a00nutstore\008\zw08\电商\阿里巴巴1688\阿里巴巴订单中没有品名(1).xlsx")
df0

,订单号,货品总价,运费（元）,商家改价（元）,实付款（元）,订单状态,订单创建时间,单品货号,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,4029296580454477008,10.55,2.8,0.0,13.35,交易成功,2024-09-03 01:21:57,EXLA5100Pa,5,NaN,NaN
1,2278067991350911471,691.20,91.2,0.0,782.40,交易成功,2024-09-02 02:27:30,EXLB5100Pa,240,NaN,NaN
2,2278724269209911471,864.00,113.3,0.0,977.30,交易关闭,2024-09-02 02:25:38,NaN,NaN,NaN,NaN
3,2278065363942911471,1440.00,187.4,0.0,1627.40,交易关闭,2024-09-02 02:24:24,NaN,NaN,NaN,NaN
4,2275086218060301182,4.86,2.8,0.0,7.66,交易成功,2024-08-30 10:30:40,EXLA6100P,3,NaN,NaN
5,2275086218060301182,0.00,0.0,0.0,0.00,交易成功,2024-08-30 10:30:40,EXBA7100P,2,NaN,NaN
6,4022437645929203749,6109.60,425.0,-1593.6,4941.00,交易成功,2024-08-28 16:53:19,XJ404S,16K精品牛卡数学缝线本24P,280.0,NaN
7,4022437645929203749,0.00,0.0,0.0,0.00,交易成功,2024-08-28 16:53:19,XJ403S,16K精品牛卡作文缝线本24P,280.0,NaN
8,4022437645929203749,0.00,0.0,0.0,0.00,交易成功,2024-08-28 16:53:19,XJ401S,16K精品牛卡英语缝线本24P,280.0,NaN
9,4022437645929203749,0.00,0.0,0.0,0.00,交易成功,2024-08-28 16:53:19,XJ412,16K精品牛卡缝线纠错本24P,280.0,NaN


In [22]:
df0 = df0.astype({'订单号':str})
df0

,订单号,货品总价,运费（元）,商家改价（元）,实付款（元）,订单状态,订单创建时间,单品货号,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,4029296580454477008,10.55,2.8,0.0,13.35,交易成功,2024-09-03 01:21:57,EXLA5100Pa,5,NaN,NaN
1,2278067991350911471,691.20,91.2,0.0,782.40,交易成功,2024-09-02 02:27:30,EXLB5100Pa,240,NaN,NaN
2,2278724269209911471,864.00,113.3,0.0,977.30,交易关闭,2024-09-02 02:25:38,NaN,NaN,NaN,NaN
3,2278065363942911471,1440.00,187.4,0.0,1627.40,交易关闭,2024-09-02 02:24:24,NaN,NaN,NaN,NaN
4,2275086218060301182,4.86,2.8,0.0,7.66,交易成功,2024-08-30 10:30:40,EXLA6100P,3,NaN,NaN
5,2275086218060301182,0.00,0.0,0.0,0.00,交易成功,2024-08-30 10:30:40,EXBA7100P,2,NaN,NaN
6,4022437645929203749,6109.60,425.0,-1593.6,4941.00,交易成功,2024-08-28 16:53:19,XJ404S,16K精品牛卡数学缝线本24P,280.0,NaN
7,4022437645929203749,0.00,0.0,0.0,0.00,交易成功,2024-08-28 16:53:19,XJ403S,16K精品牛卡作文缝线本24P,280.0,NaN
8,4022437645929203749,0.00,0.0,0.0,0.00,交易成功,2024-08-28 16:53:19,XJ401S,16K精品牛卡英语缝线本24P,280.0,NaN
9,4022437645929203749,0.00,0.0,0.0,0.00,交易成功,2024-08-28 16:53:19,XJ412,16K精品牛卡缝线纠错本24P,280.0,NaN


In [23]:
df0.dtypes

订单号                    object
货品总价                  float64
运费（元）                 float64
商家改价（元）               float64
实付款（元）                float64
订单状态                   object
订单创建时间         datetime64[ns]
单品货号                   object
Unnamed: 8             object
Unnamed: 9            float64
Unnamed: 10            object
dtype: object

In [24]:
df1 = pd.read_excel(r"F:\a00nutstore\008\zw08\电商\阿里巴巴1688\阿里巴巴202240726-20241025正 - 副本.xlsx",dtype = {'订单号':str})
df1

,订单号,买家公司名称,买家会员,卖家公司名称,卖家会员名,货品总价,运费（元）,商家改价（元）,实付款（元）,订单状态,...,买家留言,货运公司,运单号,发票：购货单位名称,发票：纳税人识别号,发票：地址、电话,发票：开户行及账号,发票：收取地址,下游订单号,下单公司主体
0,4098972747591264809,NaN,tb60477203,湖北双佳纸品有限公司,湖北双佳纸品有限公司,8.64,2.5,0.0,11.14,等待买家确认收货,...,NaN,申通快递(STO),773318114549095,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2343366266994598999,任剑（个人）天津良子轮胎,supermissiler,湖北双佳纸品有限公司,湖北双佳纸品有限公司,6.76,2.5,0.0,9.26,交易关闭,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4098081853230213312,NaN,简简单单的小幸福1995,湖北双佳纸品有限公司,湖北双佳纸品有限公司,11.80,3.0,0.0,14.80,交易关闭,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2342481888793527358,NaN,tb658008243,湖北双佳纸品有限公司,湖北双佳纸品有限公司,78.70,8.5,0.0,87.20,等待买家确认收货,...,NaN,申通快递(STO),773318114549089,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1414,2273049049602822267,NaN,尚贤增88,湖北双佳纸品有限公司,湖北双佳纸品有限公司,88.00,17.0,11.0,116.00,交易成功,...,换成16K大方格本,圆通速递(YTO),YT7488519267802,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1415,2269186502496746753,李菲（个人）,more_lion_shop,湖北双佳纸品有限公司,湖北双佳纸品有限公司,15.60,4.3,0.0,19.90,交易成功,...,NaN,圆通速递(YTO),YT7488056363210,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1416,2268563451685578186,NaN,许55555555,湖北双佳纸品有限公司,湖北双佳纸品有限公司,60.12,11.0,0.0,71.12,交易成功,...,NaN,圆通速递(YTO),YT7488055698555,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df1.dtypes

订单号                  object
买家公司名称               object
买家会员                 object
卖家公司名称               object
卖家会员名                object
货品总价                float64
运费（元）               float64
商家改价（元）             float64
实付款（元）              float64
订单状态                 object
订单创建时间       datetime64[ns]
订单付款时间       datetime64[ns]
收货人姓名                object
收货地址                 object
邮编                  float64
联系电话                 object
联系手机                 object
货品标题                 object
单价                  float64
数量                    int64
单位                   object
货号                   object
型号                   object
Offer_ID              int64
SKU_ID                int64
单品货号                 object
货品种类                float64
买家留言                 object
货运公司                 object
运单号                  object
发票：购货单位名称            object
发票：纳税人识别号            object
发票：地址、电话             object
发票：开户行及账号            object
发票：收取地址              object
下游订单号               

In [28]:
df0 = df0.set_index('订单号')
df0

,货品总价,运费（元）,商家改价（元）,实付款（元）,订单状态,订单创建时间,单品货号,Unnamed: 8,Unnamed: 9,Unnamed: 10
订单号,,,,,,,,,,
4029296580454477008,10.55,2.8,0.0,13.35,交易成功,2024-09-03 01:21:57,EXLA5100Pa,5,NaN,NaN
2278067991350911471,691.20,91.2,0.0,782.40,交易成功,2024-09-02 02:27:30,EXLB5100Pa,240,NaN,NaN
2278724269209911471,864.00,113.3,0.0,977.30,交易关闭,2024-09-02 02:25:38,NaN,NaN,NaN,NaN
2278065363942911471,1440.00,187.4,0.0,1627.40,交易关闭,2024-09-02 02:24:24,NaN,NaN,NaN,NaN
2275086218060301182,4.86,2.8,0.0,7.66,交易成功,2024-08-30 10:30:40,EXLA6100P,3,NaN,NaN
2275086218060301182,0.00,0.0,0.0,0.00,交易成功,2024-08-30 10:30:40,EXBA7100P,2,NaN,NaN
4022437645929203749,6109.60,425.0,-1593.6,4941.00,交易成功,2024-08-28 16:53:19,XJ404S,16K精品牛卡数学缝线本24P,280.0,NaN
4022437645929203749,0.00,0.0,0.0,0.00,交易成功,2024-08-28 16:53:19,XJ403S,16K精品牛卡作文缝线本24P,280.0,NaN
4022437645929203749,0.00,0.0,0.0,0.00,交易成功,2024-08-28 16:53:19,XJ401S,16K精品牛卡英语缝线本24P,280.0,NaN


In [27]:
df1 = df1.set_index('订单号')
df1

,买家公司名称,买家会员,卖家公司名称,卖家会员名,货品总价,运费（元）,商家改价（元）,实付款（元）,订单状态,订单创建时间,...,买家留言,货运公司,运单号,发票：购货单位名称,发票：纳税人识别号,发票：地址、电话,发票：开户行及账号,发票：收取地址,下游订单号,下单公司主体
订单号,,,,,,,,,,,,,,,,,,,,,
4098972747591264809,NaN,tb60477203,湖北双佳纸品有限公司,湖北双佳纸品有限公司,8.64,2.5,0.0,11.14,等待买家确认收货,2024-10-25 23:43:50,...,NaN,申通快递(STO),773318114549095,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2343366266994598999,任剑（个人）天津良子轮胎,supermissiler,湖北双佳纸品有限公司,湖北双佳纸品有限公司,6.76,2.5,0.0,9.26,交易关闭,2024-10-25 22:27:52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4098081853230213312,NaN,简简单单的小幸福1995,湖北双佳纸品有限公司,湖北双佳纸品有限公司,11.80,3.0,0.0,14.80,交易关闭,2024-10-25 17:52:33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2342481888793527358,NaN,tb658008243,湖北双佳纸品有限公司,湖北双佳纸品有限公司,78.70,8.5,0.0,87.20,等待买家确认收货,2024-10-25 17:01:17,...,NaN,申通快递(STO),773318114549089,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2273049049602822267,NaN,尚贤增88,湖北双佳纸品有限公司,湖北双佳纸品有限公司,88.00,17.0,11.0,116.00,交易成功,2024-08-28 10:03:09,...,换成16K大方格本,圆通速递(YTO),YT7488519267802,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2269186502496746753,李菲（个人）,more_lion_shop,湖北双佳纸品有限公司,湖北双佳纸品有限公司,15.60,4.3,0.0,19.90,交易成功,2024-08-24 22:34:58,...,NaN,圆通速递(YTO),YT7488056363210,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df1[df0.index.to_list()]

KeyError: "None of [Index(['4029296580454477008', '2278067991350911471', '2278724269209911471',\n       '2278065363942911471', '2275086218060301182', '2275086218060301182',\n       '4022437645929203749', '4022437645929203749', '4022437645929203749',\n       '4022437645929203749', '4022437645929203749', '4022437645929203749',\n       '4022437645929203749', '4022437645929203749', '4022437645929203749',\n       '4022437645929203749', '4022437645929203749', '4022437645929203749',\n       '4022437645929203749', '4022437645929203749', '4022631722109203749',\n       '4022631722109203749', '4022631722109203749', '2273049049602822267'],\n      dtype='object')] are in the [columns]"

In [32]:
df1[df1.index.isin(df0.index.to_list())]

,买家公司名称,买家会员,卖家公司名称,卖家会员名,货品总价,运费（元）,商家改价（元）,实付款（元）,订单状态,订单创建时间,...,买家留言,货运公司,运单号,发票：购货单位名称,发票：纳税人识别号,发票：地址、电话,发票：开户行及账号,发票：收取地址,下游订单号,下单公司主体
订单号,,,,,,,,,,,,,,,,,,,,,
4029296580454477008,NaN,manager_huangqf,湖北双佳纸品有限公司,湖北双佳纸品有限公司,10.55,2.8,0.0,13.35,交易成功,2024-09-03 01:21:57,...,NaN,圆通速递(YTO),YT7489717680501,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2278067991350911471,NaN,逍遥先生718991506,湖北双佳纸品有限公司,湖北双佳纸品有限公司,691.20,91.2,0.0,782.40,交易成功,2024-09-02 02:27:30,...,NaN,圆通速递(YTO),YT7489618782346,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2278724269209911471,NaN,逍遥先生718991506,湖北双佳纸品有限公司,湖北双佳纸品有限公司,864.00,113.3,0.0,977.30,交易关闭,2024-09-02 02:25:38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2278065363942911471,NaN,逍遥先生718991506,湖北双佳纸品有限公司,湖北双佳纸品有限公司,1440.00,187.4,0.0,1627.40,交易关闭,2024-09-02 02:24:24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2275086218060301182,NaN,tb70609875,湖北双佳纸品有限公司,湖北双佳纸品有限公司,4.86,2.8,0.0,7.66,交易成功,2024-08-30 10:30:40,...,NaN,圆通速递(YTO),YT7488971115284,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4022437645929203749,NaN,tb267607858,湖北双佳纸品有限公司,湖北双佳纸品有限公司,6109.60,425.0,-1593.6,4941.00,交易成功,2024-08-28 16:53:19,...,NaN,武汉兆通物流有限公司,0003539,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4022631722109203749,NaN,tb267607858,湖北双佳纸品有限公司,湖北双佳纸品有限公司,13.20,45.0,0.0,58.20,交易关闭,2024-08-28 16:35:45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2273049049602822267,NaN,尚贤增88,湖北双佳纸品有限公司,湖北双佳纸品有限公司,88.00,17.0,11.0,116.00,交易成功,2024-08-28 10:03:09,...,换成16K大方格本,圆通速递(YTO),YT7488519267802,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df1.index

Index([ 4.098972747591265e+18,  2.343366266994599e+18, 4.0980818532302126e+18,
        2.342481888793528e+18,                    nan, 4.0981049304573046e+18,
                          nan,   2.34215820776286e+18, 2.3423821260473626e+18,
       2.3419966441278336e+18,
       ...
                          nan,                    nan,                    nan,
        4.022631722109204e+18,                    nan,                    nan,
       2.2730490496028224e+18, 2.2691865024967468e+18,  2.268563451685578e+18,
        2.267184001595653e+18],
      dtype='float64', name='订单号', length=1418)

In [34]:
lst = df0.index.to_list()
lst

['4029296580454477008',
 '2278067991350911471',
 '2278724269209911471',
 '2278065363942911471',
 '2275086218060301182',
 '2275086218060301182',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022437645929203749',
 '4022631722109203749',
 '4022631722109203749',
 '4022631722109203749',
 '2273049049602822267']

In [37]:
lst1 = list(set(lst))
lst1

['2273049049602822267',
 '2278067991350911471',
 '4022437645929203749',
 '2278724269209911471',
 '2275086218060301182',
 '2278065363942911471',
 '4022631722109203749',
 '4029296580454477008']

In [40]:
df1[df1.index == '2275086218060301182']


,买家公司名称,买家会员,卖家公司名称,卖家会员名,货品总价,运费（元）,商家改价（元）,实付款（元）,订单状态,订单创建时间,...,买家留言,货运公司,运单号,发票：购货单位名称,发票：纳税人识别号,发票：地址、电话,发票：开户行及账号,发票：收取地址,下游订单号,下单公司主体
订单号,,,,,,,,,,,,,,,,,,,,,
2275086218060301182,NaN,tb70609875,湖北双佳纸品有限公司,湖北双佳纸品有限公司,4.86,2.8,0.0,7.66,交易成功,2024-08-30 10:30:40,...,NaN,圆通速递(YTO),YT7488971115284,NaN,NaN,NaN,NaN,NaN,NaN,NaN
